# Part 1: Text Processing and Exploratory Data Analysis

## 1. Data Preparation

In [2]:
#pip install nltk (in case you do not have nltk)
import nltk
import json
from collections import defaultdict
from array import array
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import math
import numpy as np
import collections
from numpy import linalg as la
import pandas as pd

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
#We load the products info in a dataframe
products_path = '../../data/fashion_products_dataset.json'
with open(products_path, "r", encoding="utf-8") as f:
    products = pd.read_json(products_path)

display(products.head())

,_id,actual_price,average_rating,brand,category,crawled_at,description,discount,images,out_of_stock,pid,product_details,seller,selling_price,sub_category,title,url
0,fa8e22d6-c0b6-5229-bb9e-ad52eda39a0a,"2,999",3.9,York,Clothing and Accessories,2021-02-10 20:11:51,Yorker trackpants made from 100% rich combed c...,69% off,[https://rukminim1.flixcart.com/image/128/128/...,False,TKPFCZ9EA7H5FYZH,"[{'Style Code': '1005COMBO2'}, {'Closure': 'El...",Shyam Enterprises,921,Bottomwear,Solid Women Multicolor Track Pants,https://www.flipkart.com/yorker-solid-men-mult...
1,893e6980-f2a0-531f-b056-34dd63fe912c,"1,499",3.9,York,Clothing and Accessories,2021-02-10 20:11:52,Yorker trackpants made from 100% rich combed c...,66% off,[https://rukminim1.flixcart.com/image/128/128/...,False,TKPFCZ9EJZV2UVRZ,"[{'Style Code': '1005BLUE'}, {'Closure': 'Draw...",Shyam Enterprises,499,Bottomwear,Solid Men Blue Track Pants,https://www.flipkart.com/yorker-solid-men-blue...
2,eb4c8eab-8206-59d0-bcd1-a724d96bf74f,"2,999",3.9,York,Clothing and Accessories,2021-02-10 20:11:52,Yorker trackpants made from 100% rich combed c...,68% off,[https://rukminim1.flixcart.com/image/128/128/...,False,TKPFCZ9EHFCY5Z4Y,"[{'Style Code': '1005COMBO4'}, {'Closure': 'El...",Shyam Enterprises,931,Bottomwear,Solid Men Multicolor Track Pants,https://www.flipkart.com/yorker-solid-men-mult...
3,3f3f97bb-5faf-57df-a9ff-1af24e2b1045,"2,999",3.9,York,Clothing and Accessories,2021-02-10 20:11:53,Yorker trackpants made from 100% rich combed c...,69% off,[https://rukminim1.flixcart.com/image/128/128/...,False,TKPFCZ9ESZZ7YWEF,"[{'Style Code': '1005COMBO3'}, {'Closure': 'El...",Shyam Enterprises,911,Bottomwear,Solid Women Multicolor Track Pants,https://www.flipkart.com/yorker-solid-men-mult...
4,750caa3d-6264-53ca-8ce1-94118a1d8951,"2,999",3.9,York,Clothing and Accessories,2021-02-10 20:11:53,Yorker trackpants made from 100% rich combed c...,68% off,[https://rukminim1.flixcart.com/image/128/128/...,False,TKPFCZ9EVXKBSUD7,"[{'Style Code': '1005COMBO1'}, {'Closure': 'Dr...",Shyam Enterprises,943,Bottomwear,"Solid Women Brown, Grey Track Pants",https://www.flipkart.com/yorker-solid-men-brow...


## 1.1. Pre-Processing text

In [5]:
#We create a function to pre-process a line
def build_terms(line):
    """
    Preprocess a line:
    ●  Removing stop words 
    ●  Tokenization 
    ●  Removing punctuation marks 
    ●  Stemming 
    ●  Transforming to lowercase

    Argument:
    line -- string (text) to be preprocessed

    Returns:
    line - a list of tokens corresponding to the input text after the preprocessing
    """

    stemmer = PorterStemmer()
    stop_words = set(stopwords.words("english"))
    line = line.lower()
    line = line.split()
    line = [x for x in line if x not in stop_words]
    line = [stemmer.stem(word) for word in line]
    return line

## 1.2. Information to return when doing a search 

In [6]:
def get_products_information(products_df):
    elements = ["_id", "title", "description", "brand", "category", "sub_category", 
                "product_details", "seller", "out_of_stock", "selling_price", 
                "discount", "actual_price", "average_rating", "url"]
    
    # Selecciona solo las columnas que nos interesan
    products_df = products_df[elements]
    
    return products_df

In [7]:
products = get_products_information(products)

products["processed_title"] = products["title"].apply(build_terms)
products["processed_description"] = products["description"].apply(build_terms)

In [8]:
display(products.head(5))

,_id,title,description,brand,category,sub_category,product_details,seller,out_of_stock,selling_price,discount,actual_price,average_rating,url,processed_title,processed_description
0,fa8e22d6-c0b6-5229-bb9e-ad52eda39a0a,Solid Women Multicolor Track Pants,Yorker trackpants made from 100% rich combed c...,York,Clothing and Accessories,Bottomwear,"[{'Style Code': '1005COMBO2'}, {'Closure': 'El...",Shyam Enterprises,False,921,69% off,"2,999",3.9,https://www.flipkart.com/yorker-solid-men-mult...,"[solid, women, multicolor, track, pant]","[yorker, trackpant, made, 100%, rich, comb, co..."
1,893e6980-f2a0-531f-b056-34dd63fe912c,Solid Men Blue Track Pants,Yorker trackpants made from 100% rich combed c...,York,Clothing and Accessories,Bottomwear,"[{'Style Code': '1005BLUE'}, {'Closure': 'Draw...",Shyam Enterprises,False,499,66% off,"1,499",3.9,https://www.flipkart.com/yorker-solid-men-blue...,"[solid, men, blue, track, pant]","[yorker, trackpant, made, 100%, rich, comb, co..."
2,eb4c8eab-8206-59d0-bcd1-a724d96bf74f,Solid Men Multicolor Track Pants,Yorker trackpants made from 100% rich combed c...,York,Clothing and Accessories,Bottomwear,"[{'Style Code': '1005COMBO4'}, {'Closure': 'El...",Shyam Enterprises,False,931,68% off,"2,999",3.9,https://www.flipkart.com/yorker-solid-men-mult...,"[solid, men, multicolor, track, pant]","[yorker, trackpant, made, 100%, rich, comb, co..."
3,3f3f97bb-5faf-57df-a9ff-1af24e2b1045,Solid Women Multicolor Track Pants,Yorker trackpants made from 100% rich combed c...,York,Clothing and Accessories,Bottomwear,"[{'Style Code': '1005COMBO3'}, {'Closure': 'El...",Shyam Enterprises,False,911,69% off,"2,999",3.9,https://www.flipkart.com/yorker-solid-men-mult...,"[solid, women, multicolor, track, pant]","[yorker, trackpant, made, 100%, rich, comb, co..."
4,750caa3d-6264-53ca-8ce1-94118a1d8951,"Solid Women Brown, Grey Track Pants",Yorker trackpants made from 100% rich combed c...,York,Clothing and Accessories,Bottomwear,"[{'Style Code': '1005COMBO1'}, {'Closure': 'Dr...",Shyam Enterprises,False,943,68% off,"2,999",3.9,https://www.flipkart.com/yorker-solid-men-brow...,"[solid, women, brown,, grey, track, pant]","[yorker, trackpant, made, 100%, rich, comb, co..."


## 1.3. Handle of category, sub_category, brand, product_details, and seller during pre-processing. 

In [9]:
# We join the category and sub_category into a single field
products['cat_subcat'] = products['category'] + ": " + products['sub_category']
display(products)

,_id,title,description,brand,category,sub_category,product_details,seller,out_of_stock,selling_price,discount,actual_price,average_rating,url,processed_title,processed_description,cat_subcat
0,fa8e22d6-c0b6-5229-bb9e-ad52eda39a0a,Solid Women Multicolor Track Pants,Yorker trackpants made from 100% rich combed c...,York,Clothing and Accessories,Bottomwear,"[{'Style Code': '1005COMBO2'}, {'Closure': 'El...",Shyam Enterprises,False,921,69% off,"2,999",3.9,https://www.flipkart.com/yorker-solid-men-mult...,"[solid, women, multicolor, track, pant]","[yorker, trackpant, made, 100%, rich, comb, co...",Clothing and Accessories: Bottomwear
1,893e6980-f2a0-531f-b056-34dd63fe912c,Solid Men Blue Track Pants,Yorker trackpants made from 100% rich combed c...,York,Clothing and Accessories,Bottomwear,"[{'Style Code': '1005BLUE'}, {'Closure': 'Draw...",Shyam Enterprises,False,499,66% off,"1,499",3.9,https://www.flipkart.com/yorker-solid-men-blue...,"[solid, men, blue, track, pant]","[yorker, trackpant, made, 100%, rich, comb, co...",Clothing and Accessories: Bottomwear
2,eb4c8eab-8206-59d0-bcd1-a724d96bf74f,Solid Men Multicolor Track Pants,Yorker trackpants made from 100% rich combed c...,York,Clothing and Accessories,Bottomwear,"[{'Style Code': '1005COMBO4'}, {'Closure': 'El...",Shyam Enterprises,False,931,68% off,"2,999",3.9,https://www.flipkart.com/yorker-solid-men-mult...,"[solid, men, multicolor, track, pant]","[yorker, trackpant, made, 100%, rich, comb, co...",Clothing and Accessories: Bottomwear
3,3f3f97bb-5faf-57df-a9ff-1af24e2b1045,Solid Women Multicolor Track Pants,Yorker trackpants made from 100% rich combed c...,York,Clothing and Accessories,Bottomwear,"[{'Style Code': '1005COMBO3'}, {'Closure': 'El...",Shyam Enterprises,False,911,69% off,"2,999",3.9,https://www.flipkart.com/yorker-solid-men-mult...,"[solid, women, multicolor, track, pant]","[yorker, trackpant, made, 100%, rich, comb, co...",Clothing and Accessories: Bottomwear
4,750caa3d-6264-53ca-8ce1-94118a1d8951,"Solid Women Brown, Grey Track Pants",Yorker trackpants made from 100% rich combed c...,York,Clothing and Accessories,Bottomwear,"[{'Style Code': '1005COMBO1'}, {'Closure': 'Dr...",Shyam Enterprises,False,943,68% off,"2,999",3.9,https://www.flipkart.com/yorker-solid-men-brow...,"[solid, women, brown,, grey, track, pant]","[yorker, trackpant, made, 100%, rich, comb, co...",Clothing and Accessories: Bottomwear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28075,3705c6bd-6f23-529a-9b64-15b8fc568efa,Sleeveless Solid Men Jacket,,Oka,Clothing and Accessories,Winter Wear,"[{'Color': 'Blue'}, {'Fabric': 'Nylon'}, {'Pat...",,True,"1,379",40% off,"2,299",,https://www.flipkart.com/okane-sleeveless-soli...,"[sleeveless, solid, men, jacket]",[],Clothing and Accessories: Winter Wear
28076,f2a683e4-634d-5a11-8950-2d56b924576d,Printed Men Blue Regular Shorts,,Oka,Clothing and Accessories,Bottomwear,"[{'Fabric': 'Polycotton'}, {'Pattern': 'Printe...",OKANE,False,520,34% off,799,3.8,https://www.flipkart.com/okane-printed-men-blu...,"[print, men, blue, regular, short]",[],Clothing and Accessories: Bottomwear
28077,1efa858c-1360-59a6-9624-bb04eddb492c,Checkered Single Breasted Casual Women Full Sl...,,Oka,Clothing and Accessories,"Blazers, Waistcoats and Suits","[{'Color': 'Grey'}, {'Fabric': 'Tweed'}, {'Pat...",,True,"2,999",40% off,"4,999",,https://www.flipkart.com/okane-checkered-singl...,"[checker, singl, breast, casual, women, full, ...",[],"Clothing and Accessories: Blazers, Waistcoats ..."
28078,9fdfdd22-487b-599b-8be6-5dd00eb987c5,Full Sleeve Solid Men Casual Jacket,,Oka,Clothing and Accessories,Winter Wear,"[{'Color': 'Blue'}, {'Fabric': 'Nylon'}, {'Pat...",OKANE,False,"1,875",40% off,"3,125",3.8,https://www.flipkart.com/okane-full-sleeve-sol...,"[full, sleev, solid, men, casual, jacket]",[],Clothing and Accessories: Winter Wear


## 1.4. Handle of out_of_stock, selling_price, discount, actual_price, and average_rating during pre-processing. 

## 2. Exploratory Data Analysis 